In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.



In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
print(df_train.columns)
print(df_test.columns)
print(df_train.shape, df_test.shape)
print(df_train.isnull().sum())

In [ ]:
for df in [df_train,df_test]:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']    # 침실과 욕실의 총합.
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']   # 실제 주거 공간 + 토지 전체 공간
    df['sqft_total_size'] = df['sqft_living'] + df['sqft_lot'] + df['sqft_above'] + df['sqft_basement']  # 지하실 제외 집의 평방 피트, 집 지하실 포함.
    df['sqft_total15'] = df['sqft_living15'] + df['sqft_lot15']  # 2015년 거실, 2015년 lotSize영역 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['sqft_total_by_lot'] = (df['sqft_living'] + df['sqft_above'] + df['sqft_basement'])/df['sqft_lot']
    
    #qcut_count = 10
    #df['qcut_long'] = pd.qcut(df['long'], qcut_count, labels=range(qcut_count))
    #df['qcut_lat'] = pd.qcut(df['lat'], qcut_count, labels=range(qcut_count))

### 모델 만들기

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## 새로운 변수 생성
* total_rooms = bedrooms(침실)+bathrooms(욕실)
* sqft_ration = sqft_living(실제 주거공간)/sqft_lot(토지 전체 공간)
* 

In [ ]:
y=df_train['price'] # 타깃
X=df_train.drop('price', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, #default = 75:25
                                                      random_state=123 ) 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X_train.corr()

In [ ]:
import seaborn as sns
corr=df_train.corr()
plt.subplots(figsize=(15,10))
sns.heatmap(corr, annot=True, fmt='.3f')

In [ ]:
X_train.columns

#### 데이터 변수 선택

In [ ]:
#sel = ['bedrooms', 'floors']
#sel = ['bedrooms', 'floors', 'grade', 'sqft_living'] # 2
#sel2 = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'view', 'grade', 'sqft_above',
#       'sqft_basement', 'yr_renovated','lat', 'sqft_living15'] #3
'''
X_train_sel = X_train[sel]
X_valid_sel = X_valid[sel]
X_test_sel = df_test[sel]
'''
#sel3
X_train_sel = X_train.drop('date', axis=1)
X_valid_sel = X_valid.drop('date' , axis=1)
X_test_sel=df_test.drop('date', axis=1)


In [ ]:
## 01. 모델 생성, 02. 모델 훈련, 03. 예측
model = linear_model.LinearRegression() # 생성
model.fit(X_train_sel, y_train) # 훈련
val_pred1 = model.predict(X_valid_sel) # 예측
print(val_pred1.shape)

In [ ]:
import lightgbm as lgb
from xgboost import XGBRegressor

In [ ]:
## 01. 모델2. lgbm
lightgbm = lgb.LGBMRegressor(random_state=123)
lightgbm.fit(X_train_sel, y_train)
y_pred_lgbm = lightgbm.predict(X_valid_sel)

## 01. 모델3. xgbm
xgbm_model = XGBRegressor()
xgbm_model.fit(X_train_sel, y_train)
y_pred_xgbm = xgbm_model.predict(X_valid_sel)


In [ ]:
## 모델 평가1
from math import sqrt
print('RMSE : ', sqrt(mean_squared_error(val_pred1, y_valid)))

In [ ]:
## 모델 평가2
from math import sqrt
print('RMSE : ', sqrt(mean_squared_error(val_pred1, y_valid)))

In [ ]:

print('RMSE : ', sqrt(mean_squared_error(val_pred1, y_valid)))

In [ ]:
## 모델 평가4
from math import sqrt

print('RMSE : ', sqrt(mean_squared_error(val_pred1, y_valid)))
print('RMSE(lgbm) : ', sqrt(mean_squared_error(y_pred_lgbm, y_valid)))
print('TMSE(xgbm) : ', sqrt(mean_squared_error(y_pred_xgbm, y_valid)))

In [ ]:
print(y_pred_xgbm.shape)

In [ ]:
print(sum.shape)

In [ ]:
## 01. 모델2. lgbm
lightgbm = lgb.LGBMRegressor(random_state=123)
lightgbm.fit(X_train_sel, y_train)
y_pred_lgbm = lightgbm.predict(X_test_sel)

## 01. 모델3. xgbm
xgbm_model = XGBRegressor()
xgbm_model.fit(X_train_sel, y_train)
y_pred_xgbm = xgbm_model.predict(X_test_sel)

In [ ]:
sum = pd.read_csv('../input/sample_submission.csv')
sum.head()
sum['price']=y_pred_xgbm
sum.head()

In [ ]:
sum.to_csv('1stSub.csv', index=False)

## EDA

In [ ]:
df_train.head(30)

In [ ]:
sns.distplot(df_train['price'])

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(3,2,1)
sns.boxplot(x='')